In [1]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# Check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
# Define a transform to normalize and augment the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(10),
                                transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2))])

In [4]:
# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=6, shuffle=True)

# Download and load the test data
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=6, shuffle=True)

In [5]:
# Define the network architecture
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Flatten()
        self.fc2 = nn.Linear(784, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)
    
net = Net().to(device)

In [6]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [8]:
# Train the network
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.627
[1,  4000] loss: 0.860
[1,  6000] loss: 0.784
[1,  8000] loss: 0.703
[1, 10000] loss: 0.626
[2,  2000] loss: 0.568
[2,  4000] loss: 0.506
[2,  6000] loss: 0.457
[2,  8000] loss: 0.418
[2, 10000] loss: 0.394
[3,  2000] loss: 0.362
[3,  4000] loss: 0.358
[3,  6000] loss: 0.356
[3,  8000] loss: 0.324
[3, 10000] loss: 0.315
[4,  2000] loss: 0.309
[4,  4000] loss: 0.302
[4,  6000] loss: 0.290
[4,  8000] loss: 0.283
[4, 10000] loss: 0.278
[5,  2000] loss: 0.272
[5,  4000] loss: 0.260
[5,  6000] loss: 0.253
[5,  8000] loss: 0.257
[5, 10000] loss: 0.256
[6,  2000] loss: 0.257
[6,  4000] loss: 0.228
[6,  6000] loss: 0.239
[6,  8000] loss: 0.235
[6, 10000] loss: 0.222
[7,  2000] loss: 0.223
[7,  4000] loss: 0.227
[7,  6000] loss: 0.213
[7,  8000] loss: 0.214
[7, 10000] loss: 0.226
[8,  2000] loss: 0.204
[8,  4000] loss: 0.204
[8,  6000] loss: 0.219
[8,  8000] loss: 0.211
[8, 10000] loss: 0.220
[9,  2000] loss: 0.204
[9,  4000] loss: 0.203
[9,  6000] loss: 0.193
[9,  8000] 

In [14]:
# Test the network
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        # calculate outputs by running images through the network
        outputs = net(images)
        
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {(100 * correct / total):.2f} %')

Accuracy of the network on the 10000 test images: 94.17 %
